In [2]:
# Install required libraries
!pip install PyPDF2
!pip install pymupdf

import os
import json
import fitz  # PyMuPDF
from PyPDF2 import PdfReader
from google.colab import files

# Step 1: Upload the PDF file
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Step 2: Extract text and images from the PDF
def extract_pdf_data(pdf_path):
    data = {
        "text": [],
        "images": []
    }

    # Extract text using PyPDF2
    reader = PdfReader(pdf_path)
    for page in reader.pages:
        data["text"].append(page.extract_text())

    # Extract images using PyMuPDF
    doc = fitz.open(pdf_path)
    for i in range(len(doc)):
        page = doc.load_page(i)
        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]

            # Save the image as a temporary file
            image_filename = f"page_{i+1}_img_{img_index+1}.{image_ext}"
            with open(image_filename, "wb") as img_file:
                img_file.write(image_bytes)

            data["images"].append({
                "page": i + 1,
                "image_file": image_filename
            })

    return data

# Step 3: Process the uploaded PDF file
pdf_data = extract_pdf_data(file_name)

# Step 4: Save extracted data to a JSON file
output_json_file = "extracted_data.json"
with open(output_json_file, "w") as json_file:
    json.dump(pdf_data, json_file, indent=4)

print(f"Data has been extracted and saved to {output_json_file}.")

# Step 5: Download the JSON file
files.download(output_json_file)

# Step 6: Download extracted images (optional)
for image in pdf_data["images"]:
    files.download(image["image_file"])


Saving pd.pdf to pd.pdf
Data has been extracted and saved to extracted_data.json.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>